# Práctica 2 de programación
**Atzi Merino**

Imports

In [197]:
import numpy as np
import matplotlib.pyplot as plt

Clases

In [198]:
#La clase de persona tiene los atributos que me serán útiles en los clientes
class Persona():
  def __init__(self,m_lleg,m_at,m_sal,m_esp,t):
    self.mom_llegada = m_lleg
    self.min_atencion = m_at
    self.mom_salida = m_sal
    self.min_espera = m_esp
    self.tipo_cliente = t

In [199]:
#La clase cola permite manejar listas como colas
class Cola():
  #Se puede devolver la lista
  def __init__(self):
    self.cola = []

  #Al llegar, se forman los clientes
  def enqueue(self, elemento):
    self.cola.append(elemento)

  #Al ser atendidos se irán de la cola
  def dequeue(self):
    if len(self.cola)>0:
      self.cola.pop(0)

  #Para saber si una cola tiene personas
  def vacio(self):
    if len(self.cola) == 0:
      return True
    else:
      return False

Funciones

In [200]:
#Definí una funcion para generar las lineas abiertas
#Es una lista de 0 pues indica el tiempo inicial transcurrido por linea
def lineas_abiertas(num):
  lista = []
  for i in range(num):
    lista.append(0)
  return lista

In [201]:
#Defini una función que asigna el tipo de cliente que llega dado un numero
def tipo(num):
  if 0 < num < 1/6:
    tipo = 'premier'
  else:
    tipo = 'normal'
  return tipo

In [202]:
#Usando la funcion tipo y un numero aleatorio, asigna los clientes a sus colas
#Ademas con sus respectivos atributos
def asignacion_colas(cola1, cola2, tiempos, momentos, cantidad):
  for i in range(cantidad):
    num = np.random.random()
    cliente_i = Persona(momentos[i],tiempos[i],0,0,tipo(num))
    if cliente_i.tipo_cliente == 'premier':
      cola2.append(cliente_i)
    elif cliente_i.tipo_cliente == 'normal':
      cola1.append(cliente_i)

In [203]:
#Funcion para determinar quien llego primero
#Compara el primer cliente de una cola con el primero de otra
def llego_primero(cola1, cola2):
  if cola1.cola[0].mom_llegada < cola2.cola[0].mom_llegada:
    return cola1.cola
  elif cola1.cola[0].mom_llegada > cola2.cola[0].mom_llegada:
    return cola2.cola

In [204]:
#Determina si hay una linea disponible para el primer cliente de una cola
def linea_disponible(lineas, cola):
  if lineas[0] <= cola.cola[0].mom_llegada:
    return True
  elif lineas[0] > cola.cola[0].mom_llegada:
    return False

In [205]:
#Al haber una linea disponible, hace operaciones para determinar los tiempos
def si_habia_disponible(cola, lineas, archivo):
  #Como estuvo disponible, al cliente le corresponde 0 min de espera
  cola.cola[0].min_espera = 0
  #Su hora de salida sera su hora de llegada mas sus min de atencion
  cola.cola[0].mom_salida = cola.cola[0].mom_llegada + cola.cola[0].min_atencion
  lineas[0] = cola.cola[0].mom_salida
  archivo.append(cola.cola[0]) #Se guarda el cliente en un archivo
  cola.dequeue() #Se va el cliente

In [206]:
#Al haber una linea disponible, hace operaciones para determinar los tiempos
def no_habia_disponible(cola, lineas, archivo):
  #No estuvo disponible, entonces le corresponde esperar
  #Espera la diferencia entre que llega y se desocupa la primera linea
  cola.cola[0].min_espera = lineas[0] - cola.cola[0].mom_llegada
  #Su salida es la hora en que se desocupa la linea + sus min de atencion
  cola.cola[0].mom_salida = lineas[0] + cola.cola[0].min_atencion
  lineas[0] = cola.cola[0].mom_salida
  archivo.append(cola.cola[0])
  cola.dequeue()

In [207]:
#Ejecuta como tal la simulacion. Usando las funciones anteriores
def simulacion(archivo_clientes,archivo_premier,cantidad_lineas,
               cantidad_clientes):
  #Se usan 2 tipos de cola
  normal = Cola()
  premier = Cola()
  #Se abren las lineas correspondientes dado la cantidad de lineas
  minutos_por_lineas = lineas_abiertas(cantidad_lineas)
  #Se generan lo que tardarán en atencion los clientes (1-81 min)
  tiempos_atencion = np.random.uniform(low = 1, high = 81, size =
                                       cantidad_clientes)
  #Se generan lo que tardarán en llegar los clientes (1-3 min)
  tiempos_llegadas = np.random.uniform(low = 1, high = 3,
                                       size = cantidad_clientes)
  #La suma acumulativa de los tiempos de llegada dan las horas de llegada
  momentos_de_llegadas = tiempos_llegadas.cumsum()
  #Se asignan los clientes a las colas haciendo uso de las dos listas anteriores
  asignacion_colas(normal.cola, premier.cola, tiempos_atencion,
                   momentos_de_llegadas, cantidad_clientes)
  #Aquí se empieza a trabajar con los clientes
  for i in range(cantidad_clientes):
    #Se van checando casos de colas vacías
    if not premier.vacio() and not normal.vacio():
      minutos_por_lineas.sort()
      #Se evalúa quién llegó primero
      if llego_primero(premier, normal) == premier.cola:
        #Se evalúa si hay lineas disponibles
        if linea_disponible(minutos_por_lineas, premier):
          #Si la hay se ejecuta la función ya definida
          si_habia_disponible(premier, minutos_por_lineas, archivo_premier)
        #También está el caso en que no hubo linea disponible
        else:
          #Entonces se ejecuta la funcion ya definida para este caso
          no_habia_disponible(premier, minutos_por_lineas, archivo_premier)
      elif llego_primero(premier, normal) == normal.cola:
        if linea_disponible(minutos_por_lineas, normal):
          si_habia_disponible(normal, minutos_por_lineas, archivo_clientes)
        else:
          if linea_disponible(minutos_por_lineas, premier):
            no_habia_disponible(normal, minutos_por_lineas, archivo_clientes)
          else:
            no_habia_disponible(premier, minutos_por_lineas, archivo_premier)
    elif not premier.vacio() and normal.vacio():
        minutos_por_lineas.sort()
        if linea_disponible(minutos_por_lineas, premier):
          si_habia_disponible(premier, minutos_por_lineas, archivo_premier)
        else:
          no_habia_disponible(premier, minutos_por_lineas, archivo_premier)
    elif premier.vacio() and not normal.vacio():
        minutos_por_lineas.sort()
        if linea_disponible(minutos_por_lineas, normal):
          si_habia_disponible(normal, minutos_por_lineas, archivo_clientes)
          normal.dequeue()
        else:
          no_habia_disponible(normal, minutos_por_lineas, archivo_clientes)

  #Devuelve los dos archivos con los clientes
  #Los elementos de estas listas son de tipo persona
  return archivo_premier
  return archivo_clientes

In [208]:
#Se definió la función para extraer solo los tiempos de espera de los archivos
def tiempos_espera(archivo):
  lista = []
  for i in range(len(archivo)):
    lista.append(archivo[i].min_espera)
  return lista

In [209]:
#Saca el promedio de una lista
def promedio(lista):
  contador = 0
  for i in range(len(lista)):
    contador += lista[i]
  promedio = contador/len(lista)
  return promedio


Main

In [210]:
#El main ejecuta la simulacion y las anteriores
#Recibe un numero de clientes y un numero de servidores
def main(cantidad_clientes, servidores):
  archivo_clientes = []
  archivo_premier = []
  tiempos_espera_premier = []
  simulacion(archivo_clientes, archivo_premier, servidores, cantidad_clientes)
  esperas_premier = tiempos_espera(archivo_premier)
  esperas_normal = tiempos_espera(archivo_clientes)
  promedio_espera_premier = promedio(esperas_premier)
  promedio_espera_normal = promedio(esperas_normal)
  #Devuelve los promedios de espera normales y premier
  return promedio_espera_normal, promedio_espera_premier

In [211]:
if __name__ == '__main__':
  print(main(10000,19))

(682.1717121333988, 2.275000116624861)
